In [ ]:
import math
import h5py
import numpy
from PIL import Image , ImageFilter
#import pywt
import os
import pyklb
import csv
import matplotlib.pyplot as plt
import glob

#import pyklb # this is our .klb management package
#import numpy # this imports numpy for image arraysr
import scipy # this imports all of science python. We'll probably use something from it. 
from scipy import ndimage
#import tifffile # this package is used to read the dark image. 
#import h5py
#import os
from PIL import Image , ImageFilter
from joblib import Parallel, delayed
import shutil

import javabridge
import bioformats

scanDirectory = '/mnt/tigress-CIFS/rpky/' #what directory contains the files 
#(Note - will convert subdirectories as well! - This is a feature! - Not a bug! - It's harder to do this way!)

conversionExtension = '.tif' #what file type are you looking to convert 

split_channels = True # if true, one file is made for each channel in a multichannel image. 
# If false, a single MULTI CHANNEL KLB IS CREATED. N.B. These files are valid, but may break posfai lab workflows!

njobs = 10 # this is the number of individual jobs for klb compression parallellization
numThreads= 10 # this is the number of threads pyKLB has access to when compressing individual files. 
#njobs*nthreads should probably be less than or equal to number of cores. 

##############################################
## SCRIPT STARTS HERE ########################
##############################################


files = glob.glob(scanDirectory + '/**/*' + conversionExtension, recursive=True)


path_list = []

for fullPath in files:
    path = os.path.dirname(fullPath)
    filename = os.path.basename(fullPath)
    klbname = os.path.splitext(filename)
    klbname0 = klbname[0]
    parp = 'folder_' + klbname0
    if os.path.exists(os.path.join(fullPath,'out',parp,'klbOut_' + klbname0 + '.klb')):
        print('.')
    else:
        path_list.append(fullPath)
        


timestep = 900 #timestep for aquisition in seconds 

foreground = True #is this the only thing running? If no, set to False.


doXYCrop = False #do you want to crop. 1 or True is crop, 0 or False is don't. 

    
##  enjoy the Jython below. It's not my fault. 
for path_to_file in path_list:




    javabridge.start_vm(class_path=bioformats.JARS)

    o = bioformats.OMEXML(bioformats.get_omexml_metadata(path_to_file))
    cspace = o.image().Pixels.get_SizeC()
    znum = o.image().Pixels.get_plane_count()
    zspace = o.image().Pixels.get_PhysicalSizeZ()
    timestep = o.image().Pixels.get_SizeT()
    xspace = o.image().Pixels.get_PhysicalSizeX()
    yspace = o.image().Pixels.get_PhysicalSizeY()

    #with bioformats.ImageReader.GetImageReader(path_to_file) as reader:
    #    pass
    #Remeber kids: TCZXY
    holder = numpy.zeros([timestep, cspace, o.image().Pixels.get_SizeZ(), o.image().Pixels.get_SizeX(), o.image().Pixels.get_SizeY()], dtype=numpy.uint16)
    for aye in range(0,timestep):
        #print(o.image().Pixels.get_SizeZ())
        for jay in range(0,o.image().Pixels.get_SizeZ()):
            #print(jay)
            print('.',end = '')
            image, scale = bioformats.load_image(path_to_file, z=jay, rescale=False,wants_max_intensity=True)
            if cspace > 1:
                holder[aye,:,jay,:,:] = numpy.transpose(image,[2,1,0])
            else: 
                holder[aye,:,jay,:,:] = numpy.transpose(image,[1,0])

    #print(holder.shape)
    pixel_spacing = [timestep,zspace,cspace,xspace,yspace]

    #image = numpy.transpose(image,[2 , 0 , 1])

    filename = os.path.splitext(os.path.basename(path_to_file))[0] #os.path.basename(path_to_file)
    output_path = os.path.join(os.path.dirname(path_to_file) , "folder_klbOut_" + os.path.splitext(os.path.basename(path_to_file))[0]) #os.path.dirname(path_to_file)
    if not os.path.isdir(output_path):
        os.makedirs(output_path)
        
    if split_channels:
        pixel_spacing[2] = 1; 
        for aye in range(0,cspace):
            filename2 = filename +'_ch_'+str(aye)+ '.klb'
            #print(filename2)
            filename2 = os.path.join(output_path,filename2)
            pyklb.writefull(numpy.ascontiguousarray(holder[:,[aye],:,:,:]),filename2,30,pixel_spacing,' ')
    else:
        filename = filename + '.klb'
        filename = os.path.join(output_path,filename)
        pyklb.writefull(numpy.ascontiguousarray(holder),filename,30,pixel_spacing,' ')

    print('done: file' + path_to_file)
print('done: script')
    

.....................................................................................................(1, 1, 101, 2048, 2048)
done: file/mnt/tigress-CIFS/rpky/210501_out/st2/klbOut_Cam_Long_00023.label.tif
.....................................................................................................(1, 1, 101, 2048, 2048)
done: file/mnt/tigress-CIFS/rpky/210501_out/st2/klbOut_Cam_Long_00130.label.tif
.....................................................................................................(1, 1, 101, 2048, 2048)
done: file/mnt/tigress-CIFS/rpky/210501_out/st2/klbOut_Cam_Short_00028.label.tif
.....................................................................................................(1, 1, 101, 2048, 2048)
done: file/mnt/tigress-CIFS/rpky/210501_out/st2/klbOut_Cam_Long_00093.label.tif
.....................................................................................................(1, 1, 101, 2048, 2048)
done: file/mnt/tigress-CIFS/rpky/210501_out/st2/klbOut

In [55]:
cspace

1

'/media/posfailab/Chomky_Drive1/playspace/592022'